In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn

import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [ ]:
# load data
x_train = torch.load("x_train")
x_val = torch.load("x_val")
x_test = torch.load("x_test")
y_train = torch.load("y_train")
y_val = torch.load("y_val")
y_test = torch.load("y_test")

In [ ]:
class Feedforward(torch.nn.Module):
    """
    Basic feedforward MLP for binary classification of BERT-embedding sentences
    """
    def __init__(self, input_size, hidden_size):
        super(Feedforward, self).__init__()
        self.input_size = input_size
        self.hidden_size  = hidden_size
        self.fc1 = nn.Linear(self.input_size, self.hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(self.hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
      
    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        output = self.fc2(relu)
        output = self.sigmoid(output)
        return output

In [ ]:
model = Feedforward(768, 10)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01) 
n_epochs = 10

In [ ]:
train_losses = []

model.train()

for epoch in range(n_epochs):
    train_loss = 0
    y_pred = model(x_train[:, 0, :])
    optimizer.zero_grad()
    loss = criterion(y_pred.squeeze(), y_train.squeeze())
    loss.backward()
    optimizer.step()
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    train_loss += loss.item()
    train_losses.append(train_loss)

In [ ]:
val_losses = []

model.eval() 

for epoch in range(n_epochs):
    val_loss = 0
    y_pred = model(x_val[:, 0, :])
    optimizer.zero_grad()
    loss = criterion(y_pred.squeeze(), y_val.squeeze())
    loss.backward()
    optimizer.step()
    # print('Epoch {}: val loss: {}'.format(epoch, loss.item()))
    val_loss += loss.item()
    val_losses.append(val_loss)

In [ ]:
# plot losses
plt.plot(train_losses, label="Training loss")
plt.plot(val_losses, label="Validation loss")
plt.legend()
plt.title("Losses")

In [ ]:
# evaluate predictions
test_pred = model(x_test[:, 0, :])
test_binary = np.where(test_pred > 0.5, 1, 0)
print(classification_report(test_binary, y_test.round()))